In [0]:
""" Read csv file into Colaboratory: """


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# upload dataset with school addresses
dataset = drive.CreateFile({'id': '1n5bmOjjsjZ881hQKhpZT3LQEE1hBGJtM'})
dataset.GetContentFile('school_addresses.csv')

In [0]:
""" Geocoding settings """


!pip install -U -q geopy shapely
import geopy
from geopy.exc import GeocoderServiceError
from geopy.geocoders import GoogleV3, Here
from numpy import nan
from shapely.geometry.point import Point


# Function to get the georreferenced point for a given (non null) address
def geocode(address, geocoders):
    result = nan
    if pd.notna(address):
        for geocoder in geocoders:
            try:
                response = geocoder.geocode(address)
                result = Point(response.longitude, response.latitude)
                break
            except(TypeError, AttributeError, GeocoderServiceError):
                continue
    return result

# instantiate geopy geocoders
GOOGLE_API_KEY = "#APIKEY"
HERE_APP_ID = "#APPID"
HERE_APP_CODE = "#APPCODE"
google = GoogleV3(domain = "maps.google.com.br", api_key = GOOGLE_API_KEY)
here = Here(app_id = HERE_APP_ID, app_code = HERE_APP_CODE)

# set order of preferred geocoders
geocoders = (google, here)

In [0]:
"""Geocode school addresses"""


import pandas as pd


# import local database to a dataframe
schools = pd.read_csv("school_addresses.csv")

# filter only for schools in Rio de Janeiro
schools_rio = schools[schools['CO_MUNICIPIO']==3304557]

# add geometry column of schools
schools_rio.loc[:, 'address'] = schools_rio["DS_ENDERECO"]
com_num = pd.notna(schools_rio["NU_ENDERECO"])
schools_rio.loc[com_num, 'address'] += ', ' + schools_rio["NU_ENDERECO"] 
schools_rio.loc[:, 'address'] += ' - ' + schools_rio["NO_MUNICIPIO"] + ', '+\
    schools_rio["SG_UF"] + ", Brasil"
schools_rio.loc[:, "geom"] = schools_rio["address"].apply(
    geocode, args=(geocoders,))

# delet rows with unsuccessful geocodes
schools_rio = schools_rio[pd.notna(schools_rio["geom"])]

In [0]:
""" Calculate time to schools by public transports"""


import requests


def get_routeTime(geom, lat, lon, arrival):
    coord = "{},{}".format(geom.y, geom.x)
    url = "https://route.api.here.com/routing/7.2/calculateroute.json"
    url += "?app_id=" + HERE_APP_ID + "&app_code=" + HERE_APP_CODE
    url += "&waypoint0=geo!{}{}".format(lat, lon)
    url += "&waypoint1=geo!" + coord
    url += "&combineChange=true&mode=fastest;publicTransportTimeTable"
    url += "&arrival=" + arrival
    r = requests.get(url)
    response = r.json()
    try:
        tempo = round(
            response['response']['route'][0]["summary"]["baseTime"]/60)
        return tempo
    except(KeyError):
        return

# routes from Rio de Janeiro ground zero to schools at 07a.m.
schools_rio.loc[:,"TimeDistance"] = schools_valid["geom"].apply(
    get_routeTime, args = ("-22.9190","-43.2150", "2018-12-11T07:00:00"))
schools_rio.sort_values(by="TimeDistance", inplace=True)

# delete rows with unsuccessful routings
schools_rio = schools_rio.loc[pd.notna(schools_rio["TimeDistance"])]

In [0]:
""" Save as GEOJSON """


!pip install -U -q geopandas
import geopandas as gpd


schools_georref = gpd.GeoDataFrame(schools_rio, geometry='geom')
schools_georref.to_file(
    "schools_rio.geojson", driver='GeoJSON', encoding="utf-8")